In [ ]:
import torch
import lib.utils.bookkeeping as bookkeeping
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import lib.models.models as models
import lib.models.model_utils as model_utils
from lib.datasets import mnist, maze, protein, synthetic
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils
import os
from tqdm import tqdm
from lib.datasets.maze import maze_acc
from ruamel.yaml.scalarfloat import ScalarFloat
from lib.datasets.mnist_fid import evaluate_fid_score
def categorical_log_likelihood(x, logits):
    """Log likelihood of a discretized Gaussian specialized for image data.

    Assumes data `x` consists of integers [0, num_classes-1].

    Args:
    x: where to evaluate the distribution. shape = (bs, ...), dtype=int32/int64
    logits: logits, shape = (bs, ..., num_classes)

    Returns:
    log likelihoods
    """
    pass
    #log_probs = nn.log_softmax(logits)
    #x_onehot = nn.one_hot(x, logits.shape[-1])
    #return torch.sum(log_probs * x_onehot, axis=-1)

In [ ]:
path = 'SavedModels/MNIST/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-25' # 2
config_name = 'config_001_unet14Mlogits.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_599999_unet14Mlogits.pt'

path = 'SavedModels/MNIST/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-25' # 2
config_name = 'config_001_unet.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_389999_unet.pt'

path = 'SavedModels/MNISTHollow/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2024-01-03' # 2
config_name = 'config_001_hollow.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_539999_hollow.pt'


dataset_location = 'lib/datasets'

config_path = os.path.join(path, date, config_name)
checkpoint_path = os.path.join(path, date, model_name)

cfg = bookkeeping.load_config(config_path)
cfg.data.train = False
"""
if cfg.loss.name == 'CatRM':
    cfg.loss.name = 'SDDMNLL'
elif cfg.loss.name == 'CTElbo':
    cfg.loss.name = 'TauLDRNLL'
else:
    raise KeyError("Eval not defined!")

cfg.training.train_step_name = 'EvalMNIST'
"""
device = torch.device(cfg.device)

model = model_utils.create_model(cfg, device)

optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)

state = {"model": model, "optimizer": optimizer, "n_iter": 0}

state = bookkeeping.load_state(state, checkpoint_path)
cfg.training.n_iters = 600000 
cfg.sampler.sample_freq = 100000000
cfg.saving.checkpoint_freq = 10000
cfg.sampler.num_steps = 1000

# EvalMNIST

loss = losses_utils.get_loss(cfg)

training_step = training_utils.get_train_step(cfg)

sampler = sampling_utils.get_sampler(cfg)



print("Info:")
print("--------------------------------")
print("State Iter:", state["n_iter"])
print("--------------------------------")
print("Name Dataset:", cfg.data.name)
print("Loss Name:", cfg.loss.name)

print("--------------------------------")
print("Model Name:", cfg.model.name)
print("Number of Parameters: ", sum([p.numel() for p in model.parameters()]))

model.eval()

In [ ]:
steps = [10, 20, 30, 50, 100, 250, 500, 750, 1000]
sampler_names = ['TauL' , 'LBJF'] # 'LBJF', 
#for sampler_n_i in sampler_name:
#    for step in steps:
#        data = np.load(f'unet_{sampler_n_i}{step}_mnist.npy')
for sampler_n in sampler_names:
    for step in steps:
        data = np.load(f'mnist_unet_{sampler_n}{step}.npy')
        print(data.shape)
        dataset = dataset_utils.get_dataset(cfg, device, dataset_location)

        dataloader = torch.utils.data.DataLoader(dataset,
            batch_size=data.shape[0],
            shuffle=cfg.data.shuffle)
        for true_data in tqdm(dataloader):
            print(f'mnist_unet_{sampler_n}{step}.npy')
            print("----------------------------------")
            print("Sampler:", sampler_n)
            print("Step:", step)
            fid = evaluate_fid_score(data, true_data.cpu().numpy(), 100)
            print("FID:", fid)
            break
# TauL: 10 20 30
# FID 284.3784917983221 119.89684926791293 41.04334648618652


In [ ]:
steps = np.array([10, 20, 30, 50, 100, 250, 500, 750, 1000])
fid_taul = np.array([283.87526811768316, 120.04226304249106, 40.6234648618652, 13.453524786963495, 2.6, 2.548404509903506, 2.5395556730263706, 2.48, 2.460157370950734])
fid_lbjf = np.array([367.43394813969206, 360.795573791034, 354.6901690476625, 339.59518918406286, 305.2288604477556, 212.0276736955564, 75.84530505097865, 32.956622302615955, 19.817409823054646])

In [ ]:
"""
n_rounds = 5000
nll = []

for i, minibatch in tqdm(enumerate(tqdm(dataloader))):
    minibatch = minibatch.to(device)
    l = training_step.step(state, minibatch.long(), loss)
    nll.append(l.item())

    if i >= n_rounds:
        print("Avg NLL:", np.mean(nll))
        break
# NLL: Hollow: 4.95, 4.96
# Unet: 5.15
"""   

In [ ]:
steps = np.array([10, 20, 30, 50, 100, 250, 500, 750, 1000])
fid_taul = np.array([283.87526811768316, 120.04226304249106, 40.6234648618652, 13.453524786963495, 2.6, 2.548404509903506, 2.5395556730263706, 2.48, 2.460157370950734])
fid_lbjf = np.array([367.43394813969206, 360.795573791034, 354.6901690476625, 339.59518918406286, 305.2288604477556, 212.0276736955564, 75.84530505097865, 32.956622302615955, 19.817409823054646])
plt.plot(steps, fid_taul, marker='o', color='green', label='Tau-Leaping')
plt.plot(steps, fid_lbjf, marker='o', label='Euler')
plt.xlabel('NFE', fontsize=13)
plt.ylabel('FID Value', fontsize=13)
plt.title('FID Value of Sampling Methods with Varying NFE', fontsize=14)
plt.legend()
plt.grid(True)
plt.savefig('fid.png')
plt.show()

In [ ]:
"""
steps = np.array([10, 20, 50, 100, 250, 500, 750, 1000])
mean_is_taul = np.array([2.68, 4.93, 7.63, 8.54, 8.601, 8.605, 8.64, 8.712])
std_is_taul = np.array([0.0688, 0.138, 0.123, 0.256, 0.178, 0.189, 0.166, 0.128])
mean_is_lbjf = np.array([2.2, 2.22, 2.356, 2.71,  4.366, 6.27, 6.734, 7.3])
std_is_lbjf = np.array([0.03, 0.04, 0.050, 0.065,  0.127,  0.185, 0.21, 0.204])

plt.plot(steps, mean_is_taul, marker='o', color='green', label='Tau-Leaping')
plt.fill_between(steps, [mean - std for mean, std in zip(mean_is_taul, std_is_taul)],
                 [mean + std for mean, std in zip(mean_is_taul, std_is_taul)],
                 alpha=0.3, color='green')
plt.plot(steps, mean_is_lbjf, marker='o', label='Euler')
plt.fill_between(steps, [mean - std for mean, std in zip(mean_is_lbjf, std_is_lbjf)],
                 [mean + std for mean, std in zip(mean_is_lbjf, std_is_lbjf)],
                 alpha=0.3)

plt.xlabel('NFE', fontsize=13)
plt.ylabel('Inception Score', fontsize=13)
plt.title('Inception Score of Sampling Methods with Varying NFE', fontsize=14)
plt.legend()
plt.grid(True)
#plt.savefig('inception_score.png')
plt.show()
"""

In [ ]:
"""
steps = np.array([10, 20, 50, 100, 250, 500, 750, 1000])
mean_is_taul = np.array([2.68, 4.93, 7.63, 8.601, 8.54, 8.605, 8.66, 8.712])
std_is_taul = np.array([0.0688, 0.138, 0.123, 0.256, 0.178, 0.189, 0.15, 0.128])
mean_is_lbjf = np.array([2.2, 2.22, 2.356, 2.71,  4.366, 6.27, 6.734, 7.3])
std_is_lbjf = np.array([0.03, 0.04, 0.050, 0.065,  0.127,  0.185, 0.344, 0.204])
"""

In [ ]:
# LBJF
# 10
# Inception mean:  2.198198056219918
# Inception std:  0.030151815824951578


#20
# Inception mean:  2.2240022875289394
# Inception std:  0.04002556276635036


#50
#Inception mean:  2.3566081056175205
#Inception std:  0.04870976690873823


#100
# Inception mean:  2.709736001921302
# Inception std:  0.06546383931077572

# 250
# Inception mean:  4.366223557997307
#Inception std:  0.12693698997288722

# 500
# Inception mean:  6.270140895472088
# Inception std:  0.18477733149537678

#750
#Inception mean:  6.734108512036066
#Inception std:  0.3443336055401

#1000
# Inception mean:  7.307791114561004
# Inception std:  0.20425301145961688


# TauL
# 10
#Inception mean:  2.682454029072186
#Inception std:  0.10887427294896481

#20
#Inception mean:  4.937553814295971
#Inception std:  0.13792913359753387

# 50
#Inception mean:  7.626723958044647
#Inception std:  0.12262230861087985

# 100
# Inception mean:  8.601645363936745
# Inception std:  0.25602074226782967

#250
#Inception mean:  8.54639863839709
#Inception std:  0.17805105174721006

#500
#Inception mean:  8.605389185830521
# Inception std:  0.18978970334759312

#1000
# Inception mean:  8.71253495184714
# Inception std:  0.1287123665975524


